# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

In [ ]:
from utils.helpers import get_api_key
import os

os.environ['OPENAI_API_KEY'] = get_api_key()


# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
# from langchain.tools.python.tool import PythonREPLTool
from langchain.chat_models import ChatOpenAI
# from langchain.utilities import PythonREPL
from langchain_experimental.tools import PythonREPLTool
from langchain_experimental.tools.python.tool import PythonREPL

import langchain

In [ ]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [ ]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [ ]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parse_errors=True,
    verbose=True
)

In [ ]:
langchain.debug=True
agent("What is the 25% of 300?")
langchain.debug=False

In [ ]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
langchain.debug=True
result = agent(question) 
langchain.debug=True

## Python Agent

In [ ]:
python_agent = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [ ]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [ ]:
python_agent.run(f"""
    sort these customers by last name and then first name and print the output : {customer_list}
""")

In [ ]:
import langchain
langchain.debug=True
python_agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

In [ ]:
from langchain.agents import tool
from datetime import date

In [ ]:
@tool
def time(text:str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [ ]:
custom_agent = initialize_agent(
    tools=tools+[time],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parse_errors=True,
    verbose=True
)

In [ ]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")